<a href="https://colab.research.google.com/github/DSWoodin/Pythonworkspace/blob/main/Hongong/DA/07_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 07-2 머신러닝으로 예측하기
## 모델 훈련하기

In [1]:
import gdown

gdown.download('https://bit.ly/3pK7iuu', 'ns_book7.csv', quiet=False)

Downloading...
From: https://bit.ly/3pK7iuu
To: /content/ns_book7.csv
100%|██████████| 53.8M/53.8M [00:01<00:00, 38.3MB/s]


'ns_book7.csv'

In [2]:
import pandas as pd

ns_book7 = pd.read_csv('ns_book7.csv', low_memory=False)
ns_book7.head()

,번호,도서명,저자,출판사,발행년도,ISBN,세트 ISBN,부가기호,권,주제분류번호,도서권수,대출건수,등록일자
0,1,인공지능과 흙,김동훈 지음,민음사,2021,9788937444319,NaN,NaN,NaN,NaN,1,0,2021-03-19
1,2,가짜 행복 권하는 사회,김태형 지음,갈매나무,2021,9791190123969,NaN,NaN,NaN,NaN,1,0,2021-03-19
2,3,나도 한 문장 잘 쓰면 바랄 게 없겠네,김선영 지음,블랙피쉬,2021,9788968332982,NaN,NaN,NaN,NaN,1,0,2021-03-19
3,4,예루살렘 해변,"이도 게펜 지음, 임재희 옮김",문학세계사,2021,9788970759906,NaN,NaN,NaN,NaN,1,0,2021-03-19
4,5,김성곤의 중국한시기행 : 장강·황하 편,김성곤 지음,김영사,2021,9788934990833,NaN,NaN,NaN,NaN,1,0,2021-03-19


In [3]:
from sklearn.model_selection import train_test_split

# 입력된 데이터를 무작위로 섞은 후 75%를 훈련 세트, 25%를 테스트 세트로 분할 default
# 분할 비율을 조정하려면 test_size 매개변수를 0~1 사이 값으로 지정
train_set, test_set = train_test_split(ns_book7, random_state=42)

In [4]:
print(len(train_set), len(test_set))

282577 94193


In [5]:
X_train = train_set[['도서권수']] # 열을 선택할 때 리스트로 감싸면 2차원 배열이 됨
y_train = train_set['대출건수']

print(X_train.shape, y_train.shape)
# X_train은 282,577개의 행과 1개의 열로 이루어진 데이터프레임이 됨
# 사이킷런이 입력으로는 2차원 배열, 타깃으로는 1차원 배열을 기대하기 때문에 일부러
# 사이킷런은 2차원 배열이나 행렬은 대문자로 표시하는 관례가 있음
# 따라서 2차원 배열인 X_train의 첫 글자는 대문자로 쓰고
# 1차원 배열인 y_train은 소문자로 씀

(282577, 1) (282577,)


In [6]:
from sklearn.linear_model import LinearRegression

# 모델 훈련 -> fit(입력, 타깃)
lr = LinearRegression()
lr.fit(X_train, y_train)

LinearRegression()

## 훈련된 모델을 평가하기: 결정계수

In [7]:
# 사이킷런 회귀 모델에서는 score() 메서드가 결정계수를 반환함
# 결정계수는 0~1 사이의 실수로 1에 가까울수록 타깃에 정확하게 맞는 예측에 성공했음
X_test = test_set[['도서권수']]
y_test = test_set['대출건수']

lr.score(X_test, y_test)
# 결정계수가 매우 낮음 -> 도서권수로 대출건수를 예측할 수 없음

0.10025676249337112

In [8]:
# 대출건수로 대출건수를 예측하는 모델을 만들면 결정계수가 어떠할까?
# y_train은 시리즈 객체이므로 2차원 형태의 데이터프레임으로 만들기 위해 to_frame()
lr.fit(y_train.to_frame(), y_train)
lr.score(y_test.to_frame(), y_test) # 당연히 1이 나옴

1.0

## 연속적인 값 예측하기: 선형 회귀

In [9]:
# lr 객체의 coef_속성과 intercept_속성에 학습된 선형 회귀식의 기울기와 절편이 저장됨
print(lr.coef_, lr.intercept_)
# 기울기가 1이고 절편은 0에 가까운 매우 작은 숫자 -> y = 1*x + 0 -> y = x
# 당연히 대출건수로 대출건수를 예측하는 모델이였으니 y = x임

[1.] -3.907985046680551e-14


## 카테고리 예측하기: 로지스틱 회귀

In [11]:
# 타깃이 실수가 아니라 카테고리(범주)인 경우도 잇음
# 두 개의 카테고리로 분류 -> 이진 분류, 3개 이상은 다중 분류
# 분류 알고리즘에서 타깃 카테고리를 클래스라고 부름
# 이진 분류에서 타깃은 0 or 1, 0이면 음성 클래스, 1이면 양성 클래스라고 함
# 보통 모델이 찾으려는 대상을 양성 클래스로 놓음(스펨 메일 필터링에서는 스팸 메일이 양성)
# 분류 알고리즘의 대표 알고리즘은 로지스틱 회귀
# 선형 함수의 결과값 실수 z(z=ax+b)를 로지스틱 함수(시그모이드 함수)에 통과시키면
# y는 0~1 사이의 값이 됨, 여기서 0.5 이상을 양성 클래스, 0.5 미만을 음성 클래스로 예측함
# 도서권수로 대출건수가 평균보다 높은지 아닌지를 예측하는 이진 분류 문제
borrow_mean = ns_book7['대출건수'].mean()
y_train_c = y_train > borrow_mean # 대출건수 평균보다 높으면 양성 클래스
y_test_c = y_test > borrow_mean
# y_train_c와 y_test_c는 0,1이 아니라 불리언 시리즈 객체이지만,
# 사이킷런은 False, True를 자동으로 0,1로 인식하므로 별다른 변환 없이 사용 가능

In [12]:
from sklearn.linear_model import LogisticRegression

logr = LogisticRegression()
logr.fit(X_train, y_train_c)
# 사이킷런 분류 모델에서는 score() 메서드가 정확도(accuracy)를 반환함
logr.score(X_test, y_test_c)
# 약 71%의 정확도, 양성과 음성 클래스만 있으므로 무작위 예측은 50% 일텐데,
# 71% 정도면 괜찮은 결과인가? -> 사실 y_test_c에 음성 클래스가 양성보다 훨씬 많음

0.7106154385145393

In [14]:
# 양성 클래스와 음성 클래스 분포 확인하기
y_test_c.value_counts()
# 음성 클래스가 69% 정도이고, 양성 클래스는 31% 정도
# X_test에 있는 샘플을 무조건 음성 클래스라고 예측해도 69%는 맞추게 될것임
# 이러한 불균형한 데이터에서는 71%의 정확도가 그다지 높은 값이 아님
# 데이터 셋에 있는 모든 특성을 사용하지 않았기 때문에 썩 좋은 결과를 얻지는 못함
# 또는 이 데이터 셋에 잘 맞는 다른 알고리즘이 있을 수도 있음

대출건수
False    65337
True     28856
Name: count, dtype: int64

In [16]:
from sklearn.dummy import DummyClassifier

# 사이킷런은 가장 많은 클래스로 무조건 예측을 수행하는 더미(dummy) 모델을 제공
dc = DummyClassifier()
dc.fit(X_train, y_train_c)
dc.score(X_test, y_test_c)
# 더미 모델의 정확도가 69%가 나옴 -> 분류모델을 만들 때 기준이 되는 값
# 이 값보다 낫지 않다면 유용한 모델이라고 말할 수 없음

0.6936502712515792

## 평균제곱오차(MSE)와 평균절댓값오차(MAE)로 모델 평가하기

In [17]:
lr.fit(X_train, y_train)

LinearRegression()

In [21]:
# predict() 메서드는 입력 데이터에 대한 예측을 만들어 줌
y_pred = lr.predict(X_test)

In [22]:
from sklearn.metrics import mean_absolute_error

# MAE 확인
mean_absolute_error(y_test, y_pred)
# y_test에 담긴 대출건수 평균은 11에 가까운데
# 예측이 타깃과 10이나 차이가 나는 것은 좋은 결과라고 할 수 없음
# score() 메서드 점수가 낮은 것에도 미루어 짐작했듯,
# 도서권수는 대출건수를 예측하는 데 좋은 특성이 아니라는 것을 MAE를 통해 한번 더 확인

10.35809175285386

In [23]:
y_test.mean()

11.577558841952163